In [2]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os


def biner(edges,bin_widths,histogram):
    if (len(edges)+1!=len(bin_widths)):
        print("Check edges and bin widths array sizes!")
        return
    
    bins=[]
    first_bin = histogram.GetXaxis().GetBinLowEdge(1)
    print(first_bin)
    last_bin = histogram.GetXaxis().GetBinUpEdge(histogram.GetNbinsX())
    print(last_bin)
    for i in range(0,len(edges)):
        n_spaces = int((edges[i] - first_bin)/bin_widths[i])
        bins = np.concatenate((bins,np.linspace(first_bin,edges[i],n_spaces,endpoint=False)))
        first_bin = edges[i]
        if edges[i]==edges[-1]:
            n_spaces = int((last_bin - edges[i])/bin_widths[i+1])
            bins = np.concatenate((bins,np.linspace(edges[i],last_bin,n_spaces,endpoint=False)))
            bins = np.concatenate((bins,[last_bin]))
    return bins

In [29]:
signal_sample_zll = "Signal_Sherpa.root"
samples_zll = {"Data":["Data.root"],"Signal":[signal_sample_zll],"VV":["VV.root"],"W":["Wjets.root"],"Z":["Ztautau_PoPy.root"],"ttbar":["ttbar.root"],"SingleTop":["singletop.root"],}
path_zll = "../MuMu/Plots/V26.Binned/"
histogram_zll = "tau_pt_topo_cuts"
binning=[[77,97],[77,20,103]]

File = r.TFile.Open(path_zll+samples_zll["Data"][0])
C_factor_zll = File.Get(histogram_zll)
C_factor_zll.SetDirectory(0)
File.Close()

for s in samples_zll:
    if s!="Data" and s!="Signal":
        File = r.TFile.Open(path_zll+samples_zll[s][0])
        hist = File.Get(histogram_zll)
        hist.SetDirectory(0)

        print(C_factor_zll.Integral())
        C_factor_zll.Add(hist,-1)
        print(C_factor_zll.Integral())
        File.Close()

rebining=biner(binning[0],binning[1],C_factor_zll)
nb=len(rebining)-1
C_factor_zll=C_factor_zll.Rebin(nb,histogram_zll,rebining)

File = r.TFile.Open(path_zll+samples_zll["Signal"][0])
hist = File.Get(histogram_zll)
hist=hist.Rebin(nb,histogram_zll,rebining)

C_factor_zll.Divide(hist)

File.Close()

canvas = r.TCanvas("Canvas")
canvas.cd()

pad1 = r.TPad (" pad1 "," pad1 " ,0.05 ,0.05 ,1 ,1)
pad1.SetTopMargin(0.03)
pad1.SetRightMargin(0.03)
pad1.SetLeftMargin(0.08)
pad1.SetBottomMargin(0.0)
pad1.Draw ()
pad1.cd ()

C_factor_zll.Draw("pe same")


canvas.Print("SFs.pdf")

370949.0
364168.99005126953
364168.99005126953
364168.99005126953
364168.99005126953
364090.6000366211
364090.6000366211
363552.56311035156
363552.56311035156
363475.7680358887
0.0
200.0


Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas
Info in <TCanvas::Print>: pdf file SFs.pdf has been created


In [19]:
C_factor_zll.GetBinContent(200)

0.9616692662239075